# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

In [1]:
from collections import Counter
import multiprocessing as mp
import csv

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

In [2]:
%%file count_letters.py
from collections import Counter

def count_letters(file):
    with open(file, 'r') as fp:
        return Counter(fp.read().lower())

Overwriting count_letters.py


In [3]:
from count_letters import count_letters

In [4]:
count_letters('Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt')
count_letters('Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt')

Counter({'с': 50084,
         'п': 25652,
         'а': 73555,
         'и': 62030,
         'б': 16016,
         'о': 106740,
         ',': 26973,
         ' ': 182305,
         'ч': 16492,
         'т': 59813,
         'к': 30802,
         'л': 42328,
         'н': 60920,
         'г': 16174,
         'у': 27309,
         'в': 43700,
         'е': 80972,
         'й': 9747,
         'э': 3203,
         'р': 39784,
         'b': 25,
         'o': 104,
         'k': 16,
         's': 96,
         'c': 42,
         'a': 98,
         'f': 23,
         'e': 162,
         '.': 9864,
         'n': 114,
         't': 98,
         ':': 984,
         'h': 48,
         'p': 29,
         '/': 22,
         '\n': 8583,
         'u': 86,
         'r': 76,
         'd': 38,
         'v': 65,
         'i': 235,
         'y': 5,
         '_': 8,
         '-': 3558,
         '1': 384,
         '0': 110,
         '9': 100,
         '6': 271,
         'm': 54,
         'l': 46,
         'ж': 10552,
     

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

In [5]:
if __name__ == '__main__':
    files = ['Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt',
            'Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt']
    with mp.Pool(processes=len(files)) as pool:
        counters = pool.map(count_letters, files)

In [6]:
counters

[Counter({'с': 11507,
          'п': 5489,
          'а': 18236,
          'и': 13587,
          'б': 3980,
          'о': 23130,
          ',': 6372,
          ' ': 45076,
          'ч': 4113,
          'т': 14245,
          'к': 6744,
          'л': 9961,
          'н': 14240,
          'г': 3948,
          'у': 6044,
          'в': 9398,
          'е': 20054,
          'й': 2028,
          'э': 836,
          'р': 9482,
          'b': 220,
          'o': 377,
          'k': 21,
          's': 429,
          'c': 324,
          'a': 590,
          'f': 52,
          'e': 1200,
          '.': 2954,
          'n': 459,
          't': 332,
          ':': 212,
          'h': 227,
          'p': 100,
          '/': 20,
          '\n': 2734,
          'u': 285,
          'r': 308,
          'd': 192,
          'v': 87,
          'i': 369,
          'y': 8,
          '_': 4,
          '-': 900,
          '1': 46,
          '0': 22,
          '9': 36,
          '6': 42,
          'm': 401,
 

## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

Фрагмент одного из файлов, которые должны получиться в результате:
```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [7]:
n_lines = 2300000
n_files = 8
with open('recipes_full.csv', encoding='utf-8') as fp:
    reader = csv.DictReader(fp)
    n = 0
    k = 0
    line = next(reader)
    while(line != {}):
        if k == 0:
            n += 1 
            filename = f'id_tag_nsteps_{n}.csv'
            f = open(filename, 'w', newline='')
            writer = csv.writer(f, delimiter=';')
        for tag in line['tags'].strip('][').split(', '):
            if tag.strip("'") != '':
                writer.writerow([line['id'], tag.strip("'"), line['n_steps']])
        k += 1
        if k == round(n_lines/n_files):
            f.close()
            k = 0
        try:
            line = next(reader)
        except StopIteration:
            line = {}
    f.close()

2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [8]:
%%file mean_steps_for_tags.py
import csv

def mean_steps_for_tags(file):
    d = {}
    with open(file, 'r') as fp:
        reader = csv.reader(fp, delimiter=';')
        for line in reader:
            if line[1] not in d.keys():
                d[line[1]] = [0, 0]
            d[line[1]][0] += int(line[2])
            d[line[1]][1] += 1
    return dict(map(lambda k: (k, d[k][0]/d[k][1]), d.keys()))

Writing mean_steps_for_tags.py


In [9]:
from mean_steps_for_tags import mean_steps_for_tags

In [10]:
if __name__ == '__main__':
    files = [f'id_tag_nsteps_{i}.csv' for i in range(1, 9)]
    with mp.Pool(processes=len(files)) as pool:
        mean_steps = pool.map(mean_steps_for_tags, files)

In [11]:
mean_steps[0]

{'mexican': 5.311320754716981,
 'healthy-2': 6.332820775670596,
 'orange-roughy': 3.4480574324324325,
 'chicken-thighs-legs': 4.181891348088532,
 'freezer': 3.919798527702441,
 'whitefish': 3.526072329688814,
 'pork-sausage': 4.298139004937334,
 'brunch': 6.906736842105263,
 'ham-and-bean-soup': 3.5179180887372015,
 'colombian': 3.5746367239101717,
 'savory-pies': 4.291799363057325,
 'refrigerator': 4.748078266946192,
 'australian': 4.24545116969922,
 'served-cold': 4.941708229426434,
 'spaghetti': 4.131141045958795,
 'passover': 3.6275109170305675,
 'quick-breads': 4.967139175257732,
 'californian': 3.75273168757588,
 'namibian': 3.5004244482173177,
 'candy': 4.291462955998496,
 'independence-day': 4.142910447761194,
 'baking': 3.5878839590443685,
 'pennsylvania-dutch': 3.5395445134575567,
 'weeknight': 7.436413361984763,
 '60-minutes-or-less': 9.320880644881685,
 'time-to-make': 9.247593582887701,
 'course': 9.237038000195039,
 'cuisine': 9.12648110495688,
 'preparation': 9.289580430

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


In [12]:
def merge_dicts(dicts):
    keys = set()
    for dictt in dicts:
        keys = keys|dictt.keys()
    d = {}
    for k in keys:
        d[k] = [0, 0]
        for dictt in dicts:
            if k in dictt.keys():  
                d[k][0] += dictt[k][0]
                d[k][1] += dictt[k][1]
    return d

In [13]:
def mean_steps_for_file():
    result = {}
    for i in range(1, 9):
        with open(f'id_tag_nsteps_{i}.csv' , 'r') as fp:
            d = {}
            reader = csv.reader(fp, delimiter=';')
            for line in reader:
                if line[1] not in d.keys():
                    d[line[1]] = [0, 0]
                d[line[1]][0] += int(line[2])
                d[line[1]][1] += 1
        result = merge_dicts((result, d))
    return dict(map(lambda k: (k, result[k][0]/result[k][1]), result.keys()))

In [14]:
%%time
mean_steps_for_file()

Wall time: 25.1 s


{'tempeh': 3.527434613244296,
 'meat': 8.918907625325192,
 'to-go': 6.433847655720368,
 'english': 4.146556529671109,
 'ham': 4.030267694179679,
 'yeast': 5.179112222274636,
 'chick-peas-garbanzos': 3.819497053872054,
 '4-hours-or-less': 10.065202927478376,
 'italian': 5.87388405910101,
 'plums': 3.5635716242125444,
 'lobster': 3.59012222775289,
 'stove-top': 7.2551772029463555,
 'breakfast': 6.098461827636979,
 'pizza': 4.149139288071904,
 'moroccan': 3.8259548383716666,
 'jewish-sephardi': 3.5652769407768585,
 'presentation': 7.177143929442672,
 'birthday': 3.6247259973695747,
 'zucchini': 3.58697079094191,
 'duck': 3.5997391020763128,
 'rosh-hashana': 3.6900091857135138,
 'vietnamese': 3.6583629893238436,
 'dinner-party': 8.233824137497079,
 'veggie-burgers': 3.512299168975069,
 'yams-sweet-potatoes': 4.005262620069487,
 'japanese': 3.788570832681579,
 'cuban': 3.5884699453551914,
 'ramadan': 3.6026196087028004,
 'chinese': 4.309035268148297,
 'chicken-breasts': 5.757057608446178,
 

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [15]:
%%file mean_steps_for_file_mp.py
import csv

def mean_steps_for_file_mp(file):
    with open(file, 'r') as fp:
        d = {}
        reader = csv.reader(fp, delimiter=';')
        for line in reader:
            if line[1] not in d.keys():
                d[line[1]] = [0, 0]
            d[line[1]][0] += int(line[2])
            d[line[1]][1] += 1
    return d 

Writing mean_steps_for_file_mp.py


In [16]:
from mean_steps_for_file_mp import mean_steps_for_file_mp

In [17]:
%%time

if __name__ == '__main__':
    files = [f'id_tag_nsteps_{i}.csv' for i in range(1, 9)]
    with mp.Pool(processes=len(files)) as pool:
        dict_steps = pool.map(mean_steps_for_file_mp, files)
    d = merge_dicts(dict_steps)
    result = dict(map(lambda k: (k, d[k][0]/d[k][1]), d.keys()))

Wall time: 7.63 s


In [18]:
result

{'tempeh': 3.527434613244296,
 'meat': 8.918907625325192,
 'to-go': 6.433847655720368,
 'english': 4.146556529671109,
 'ham': 4.030267694179679,
 'yeast': 5.179112222274636,
 'chick-peas-garbanzos': 3.819497053872054,
 '4-hours-or-less': 10.065202927478376,
 'italian': 5.87388405910101,
 'plums': 3.5635716242125444,
 'lobster': 3.59012222775289,
 'stove-top': 7.2551772029463555,
 'breakfast': 6.098461827636979,
 'pizza': 4.149139288071904,
 'moroccan': 3.8259548383716666,
 'jewish-sephardi': 3.5652769407768585,
 'presentation': 7.177143929442672,
 'birthday': 3.6247259973695747,
 'zucchini': 3.58697079094191,
 'duck': 3.5997391020763128,
 'rosh-hashana': 3.6900091857135138,
 'vietnamese': 3.6583629893238436,
 'dinner-party': 8.233824137497079,
 'veggie-burgers': 3.512299168975069,
 'yams-sweet-potatoes': 4.005262620069487,
 'japanese': 3.788570832681579,
 'cuban': 3.5884699453551914,
 'ramadan': 3.6026196087028004,
 'chinese': 4.309035268148297,
 'chicken-breasts': 5.757057608446178,
 

5. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди `multiprocessing.queue` передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 

In [19]:
%%file worker.py
from mean_steps_for_file_mp import mean_steps_for_file_mp

def worker(in_q, out_q):
    for task in iter(in_q.get, 'STOP'):
        d = mean_steps_for_file_mp(task)
        out_q.put(d)

Writing worker.py


In [20]:
from worker import worker

In [25]:
%%time
import time

if __name__ == '__main__':
    n = 2
    k = 8
    files_queue = mp.Queue()
    result_queue = mp.Queue()
    for i in range(1, k+1):
        files_queue.put(f'id_tag_nsteps_{i}.csv')
    for _ in range(n):
        files_queue.put('STOP')
    for _ in range(n):
        mp.Process(target=worker, args=(files_queue, result_queue)).start()  
    while (result_queue.qsize() != 8):
        time.sleep(.001)
    d = merge_dicts([result_queue.get() for _ in range(k)])
    result = dict(map(lambda k: (k, d[k][0]/d[k][1]), d.keys()))

Wall time: 14.1 s


In [26]:
result

{'tempeh': 3.527434613244296,
 'meat': 8.918907625325192,
 'to-go': 6.433847655720368,
 'english': 4.146556529671109,
 'ham': 4.030267694179679,
 'yeast': 5.179112222274636,
 'chick-peas-garbanzos': 3.819497053872054,
 '4-hours-or-less': 10.065202927478376,
 'italian': 5.87388405910101,
 'plums': 3.5635716242125444,
 'lobster': 3.59012222775289,
 'stove-top': 7.2551772029463555,
 'breakfast': 6.098461827636979,
 'pizza': 4.149139288071904,
 'moroccan': 3.8259548383716666,
 'jewish-sephardi': 3.5652769407768585,
 'presentation': 7.177143929442672,
 'birthday': 3.6247259973695747,
 'zucchini': 3.58697079094191,
 'duck': 3.5997391020763128,
 'rosh-hashana': 3.6900091857135138,
 'vietnamese': 3.6583629893238436,
 'dinner-party': 8.233824137497079,
 'veggie-burgers': 3.512299168975069,
 'yams-sweet-potatoes': 4.005262620069487,
 'japanese': 3.788570832681579,
 'cuban': 3.5884699453551914,
 'ramadan': 3.6026196087028004,
 'chinese': 4.309035268148297,
 'chicken-breasts': 5.757057608446178,
 